In [1]:
!pip install -r requirements.txt

In [2]:
apikey = "tJl01ofHyVljUR9Zpj444bDQ07REEpVXRdx1kpRl"

In [29]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(diet_min.index)]

    Amax = Aall.loc[Aall.index.intersection(diet_max.index)]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([diet_min,
                   -diet_max]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

In [88]:
SHEETs = [("https://docs.google.com/spreadsheets/d/17XCCM7-_Vk2erOKQmZsOZ2fYSsPUGzbGyTYZd_02bVk/edit#gid=0","Berkeley Bowl")]
SHEETs2 = [("https://docs.google.com/spreadsheets/d/17XCCM7-_Vk2erOKQmZsOZ2fYSsPUGzbGyTYZd_02bVk/edit#gid=0","Safeway")]
SHEETs3 = [("https://docs.google.com/spreadsheets/d/17XCCM7-_Vk2erOKQmZsOZ2fYSsPUGzbGyTYZd_02bVk/edit#gid=0","Whole Foods")]
SHEETs4 = [("https://docs.google.com/spreadsheets/d/17XCCM7-_Vk2erOKQmZsOZ2fYSsPUGzbGyTYZd_02bVk/edit#gid=0","Trader Joe's")]

In [89]:
import pandas as pd
from eep153_tools.sheets import read_sheets

df_bb = read_sheets(SHEETs[0][0])[SHEETs[0][1]]
df_sw = read_sheets(SHEETs2[0][0])[SHEETs2[0][1]]
df_wf = read_sheets(SHEETs3[0][0])[SHEETs3[0][1]]
df_tj = read_sheets(SHEETs4[0][0])[SHEETs4[0][1]]

Key available for students@eep153.iam.gserviceaccount.com.
Key available for students@eep153.iam.gserviceaccount.com.
Key available for students@eep153.iam.gserviceaccount.com.
Key available for students@eep153.iam.gserviceaccount.com.


In [90]:
df_bb

,Food Group,Food,Quantity,Units,Price,Date,Location,Brand Name,FDC,Vegan (1/0),Vegetarian (1/0),Pescetarian (1/0)
0,Protein,Chicken breast,1.000000,lbs,$3.99,NaN,NaN,NaN,2092152,0,0,0.0
1,Protein,Chicken thigh,1.000000,lbs,$3.99,NaN,NaN,NaN,1899680,0,0,0.0
2,Protein,Ground Beef,1.000000,lbs,$5.49,NaN,NaN,NaN,1597053,0,0,0.0
3,Protein,Eggs,672.000000,g,$3.99,NaN,NaN,NaN,577532,0,1,1.0
4,Protein,Peanut Butter,1.000000,lbs,$3.69,NaN,NaN,NaN,1100559,1,1,1.0
5,Protein,Chickpeas,1.000000,lbs,$3.99,NaN,NaN,NaN,1100433,1,1,1.0
6,Protein,Tofu,1.000000,lbs,$1.69,NaN,NaN,NaN,1864416,1,1,1.0
7,Protein,Pork Loin (Pork Loin Chop),1.000000,lbs,$7.99,NaN,NaN,NaN,168314,0,0,0.0
8,Protein,Ground Pork,1.000000,lbs,$5.99,NaN,NaN,NaN,1887701,0,0,0.0
9,Protein,Ground Turkey,1.000000,lbs,$6.49,NaN,NaN,NaN,1626048,0,0,0.0


In [68]:
# do NOT continuously run next cell

In [45]:
##do not rerun this cell while making changes to other cells, just run once. It takes like 5 minutes to run
import fooddatacentral as fdc
import warnings

D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)

In [91]:
def change_df_col_to_int(df, col):
    df[col] = df[col].dropna()
    df[col] = df[col].str.replace('$', '')
    df[col] = df[col].astype(float)
    return df

In [92]:
change_df_col_to_int(df_bb, 'Price')
change_df_col_to_int(df_sw, 'Price')
change_df_col_to_int(df_wf, 'Price')
change_df_col_to_int(df_tj, 'Price')

/tmp/ipykernel_80/3382762124.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('$', '')


,Food Group,Food,Quantity,Units,Price,Date,Location,Brand Name,FDC,Vegan (1/0),Vegetarian (1/0),Pescetarian (1/0)
0,Protein,Chicken breast,1.0000,lbs,6.99,02/27/22,Trader Joes,Trader Joes,2092152,0,0,0
1,Protein,Chicken thigh,1.0000,lbs,4.49,02/27/22,Trader Joes,Trader Joes,1899680,0,0,0
2,Protein,Ground Beef,1.0000,lbs,6.99,02/27/22,Trader Joes,Trader Joes,1597053,0,0,0
3,Protein,Eggs,672.0000,g,2.69,02/27/22,Trader Joes,Trader Joes,577532,0,1,1
4,Protein,Peanut Butter,16.0000,oz,1.99,02/27/22,Trader Joes,Trader Joes,1100559,1,1,1
5,Protein,Chickpeas,15.5000,oz,0.79,02/27/22,Trader Joes,Trader Joes,1100433,1,1,1
6,Protein,Tofu,14.0000,oz,1.79,02/27/22,Trader Joes,Trader Joes,1864416,1,1,1
7,Protein,Pork Loin,1.0000,lbs,7.99,02/27/22,Trader Joes,Trader Joes,168314,0,0,0
8,Protein,Ground Pork,16.0000,oz,5.99,02/27/22,Trader Joes,Trader Joes,1887701,0,0,0
9,Protein,Ground Turkey,16.0000,oz,3.49,02/27/22,Trader Joes,Trader Joes,1626048,0,0,0


In [98]:
# Convert food quantities to FDC units
def find_unit_prices(df):
    df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
    df['FDC Price'] = df['Price']/df['FDC Quantity']

    df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
    Prices = df.groupby('Food',sort=False)['FDC Price'].min()
    return Prices

In [100]:
prices_bb = find_unit_prices(df_bb)
prices_sw = find_unit_prices(df_sw)
prices_wf = find_unit_prices(df_wf)
prices_tj = find_unit_prices(df_tj)

In [101]:
from eep153_tools.sheets import read_sheets

DRI_url = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


In [113]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,prices_bb,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet at Berkeley Bowl for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet at Berkeley Bowl for M 19-30 is $5.90 per day.


Diet (in 100s of grams or milliliters):
Peanut Butter     1.252637
Black Beans       0.727011
Sliced Turkey     0.154066
Cheddar cheese    1.682770
Bananas           3.103034
Jalepenos         6.646953
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2778.581525          2400.0
Protein                           96.088178            56.0
Fiber, total dietary              44.420968            33.6
Folate, DFE                      400.000000           400.0
Calcium, Ca                     1451.566650          1000.0
Carbohydrate, by difference      191.411726           130.0
Iron, Fe                           8.155627             8.0
Magnesium, Mg                    444.160251           400.0
Niacin                            29.030871            16.0
Phosph

In [114]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,prices_sw,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet at Safeway for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet at Safeway for M 19-30 is $4.17 per day.


Diet (in 100s of grams or milliliters):
Peanut Butter      3.607849
Chickpeas          0.000002
Black Beans        0.003748
Pinto Beans        0.173419
Sliced Turkey      0.214842
Tuna               0.522255
Almond Milk        2.944323
Oat Milk           0.908684
Potatoes           4.090345
Romaine Lettuce    1.690839
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2699.303608          2400.0
Protein                          114.112526            56.0
Fiber, total dietary              33.600003            33.6
Folate, DFE                      400.000008           400.0
Calcium, Ca                     1000.000068          1000.0
Carbohydrate, by difference      178.874700           130.0
Iron, Fe                          13.719934             8.0
Magnesium, Mg 

In [115]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,prices_wf,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet at Whole Foods for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet at Whole Foods for M 19-30 is $5.65 per day.


Diet (in 100s of grams or milliliters):
Peanut Butter     1.658280
Chickpeas         0.648133
Black Beans       0.012525
Sliced Turkey     0.318226
Almond Milk       5.928579
Cheddar cheese    1.063297
Bananas           8.068819
Broccoli          0.158821
Spinach           0.195205
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000007          2400.0
Protein                           85.410140            56.0
Fiber, total dietary              36.286390            33.6
Folate, DFE                      400.000000           400.0
Calcium, Ca                     2043.341393          1000.0
Carbohydrate, by difference      250.108059           130.0
Iron, Fe                           9.324228             8.0
Magnesium, Mg                    602.208403    

In [116]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,prices_tj,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet at Trader Joe's for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet at Trader Joe's for M 19-30 is $4.50 per day.


Diet (in 100s of grams or milliliters):
Peanut Butter      1.873266
Chickpeas          1.129695
Black Beans        1.605966
Sliced Turkey      0.240180
Cheddar cheese     1.051570
Oat Milk           0.649234
Cauliflower        1.570837
Romaine Lettuce    1.528228
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000072          2400.0
Protein                          122.582203            56.0
Fiber, total dietary              48.685268            33.6
Folate, DFE                      400.000001           400.0
Calcium, Ca                     1313.393832          1000.0
Carbohydrate, by difference      185.833265           130.0
Iron, Fe                          14.777537             8.0
Magnesium, Mg                    444.764233           400.0
Niaci